<a href="https://colab.research.google.com/github/lmoss/onesharp/blob/main/mu/T_predicate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title
!python -m pip install -U setuptools
!python -m pip install -U git+https://github.com/lmoss/onesharp.git@main
from onesharp.interpreter.with_traces import *

# The T predicate




If someone claims that $[\![p]\!](x)\!\!\downarrow$, what evidence could they bring to prove their point?   In this section we develop an answer.  We provide a notion of a *trace* of a computation.  Here are the main points.   First, traces are finite words; they cannot be infinite objects.  Second, there is a total computable relation $T(p,x,z)$ such that if $[\![p]\!](x)\!\!\downarrow$,
then there is some $z$ such that $T(p,x,z)$.  But if $[\![p]\!](x)\!\!\uparrow$, then there is no such $z$.  This relation $T(p,x,z)$ is called the *T-predicate*.

Carrying out the definition of $T(p,x,z)$ requires us to make a number of secondary definitions.  Some of the discussion is technical because we need to code sequences of words by single words.   Please do not get bogged down by the technical details; the ideas are more important, and they are actually easier to grasp without the details of the coding.  In other words, the details are important, but they should not get in the way of understanding the ideas.

The foregoing paragraphs were about computation in ```1#```.   After we do this work, we want to port the discussion from words to numbers.  This again leads us to a mountain of details about coding.  We show that the numerical version of $T$ is primitive recursive.   This leads to the a real result proved using the $T$-predicate, the fact that every partial $\mu$-recursive function of
numbers is ```1#```-computable.

## Encoding sequences of words as words


:::{prf:definition}
:label: def-encode-sequences

We start by defining a specific encoding map

$$
c: Words^* \to Words
$$

Here is how our map words first, we define a map

$$
b : Words \to Words
$$

by 

$$
b(a_1 a_2 \cdots a_k)  =  {\tt 1}a_1 {\tt 1} a_2 \cdots  {\tt 1}a_k 
$$

In other words, we put a ${\tt 1}$ before each letter in the input word $a_1 a_2 \cdots a_k$.

Then for a sequence of words we take 

$$
c(w_1, \ldots, w_r) = {\tt 1} b(w1){\tt \#\#} b(w_2)\cdots b(w_r)
$$
 
We understand that $c(\varepsilon) = {\tt \#\#}$.
:::

Here are some Python programs about this encoding.

In [ ]:
import math

def log2int(x):
  return(math.frexp(x)[1] - 1)

def length(n):
  return(log2int(n+1))

def index(n,m): ## this is what I write as (n)_m
  a = (n+1)%(2**(m+1))
  b = 2**m
  if a < b:
    return(0)
  else:
    return(1)

def convert_numeral_to_onesharp(x):
   if x == 0:
     return('#')
   else:
     return('1')    

def s(n):
  k = length(n)  
  s = [convert_numeral_to_onesharp(index(n,k-i-1)) for i in range(k)] 
  t = "".join(s) 
  return(t)

def s_inverse(w):
  n = len(w)
  a = (2**n - 1)
  b = [(2** (n-i-1)) for i in range(n) if (w[i] == '1')]
  c = sum(b)
  d = c+a
  return(d)


In [ ]:
s(133)

In [ ]:
s_inverse('11###111#1')

## Encoding snapshots

Recall that a snapshot is a number together with a finite sequence of words.    We associate snapshots to executions of a program on some inputs; a snapshot is determined by a line number in a program, and with the contents of some finite set of registers.
But we want a definition of a snapshot encoding that does not refer to a program or to anything else.   

:::{prf:definition}
:label: def-snapshot

An *encoded snapshot* is a word of the form 

```1```${}^k$ $c(w_1, w_2, \ldots, w_{\ell})$

That is, it is a finite sequence of ```1```s followed by ```##``` followed by $b(w_1)$ for some word $w_1$, then ```###```, then 
$b(w_2)$ for some word $w_2$, $\ldots$, and finally $b(w_{\ell})$ for some word $w_{\ell}$.

:::



<img src="https://github.com/lmoss/onesharp/blob/main/harp.jpg?raw=1" width="200" height="160">

## New interface

Here is a new interface for running program which includes a "mode" capability that includes the option of seeing all the snapshots and the overall trace.

In [ ]:
step_by_step_with_snapshots('1#1#',['##'])


### We also want to check that various relations leading up to the T predicate are computable.  This part is work in progress, with the results obtained so far shown below.

In [ ]:
snapshot_check('111##1#11#1111##')

False

In [ ]:
print([0,1,2,3,4,5][2:])
print(parse('1#1#'))
halting_snapshot_check('1#1#','111##1#1#1111##1###')

[2, 3, 4, 5]
['1#', '1#']


False

<img src="https://github.com/lmoss/onesharp/blob/main/pianotrumpet.jpg?raw=1" width="200" height="160">

In [ ]:
a = ['1#','','1','','']
first_index = a.index('')
reversed_list = a[::-1]
first_index_in_reversed = reversed_list.index('')
last_index = len(a) -1 - first_index_in_reversed
last_index
print(a[:last_index-1])

['1#', '', '1']


In [ ]:
a = ['1#1#', '11', '','#','']
print(a)
first_index = a.index('')
reversed_list = a[::-1]
print(reversed_list)
first_index_in_reversed = reversed_list.index('')
last_index = len(a) -1 - first_index_in_reversed
print(last_index)
non_empty_registers = a[:last_index]
print(non_empty_registers)

['1#1#', '11', '', '#', '']
['', '#', '', '11', '1#1#']
4
['1#1#', '11', '', '#']


## Shortlex ordering of words

Our next order of business is to rework everything which we have seen from words to numbers.   This involves an encoding.  So we start with a bijection $s: N \to Words$.  

| $n$     | $s(n)$ |
| ------- | ----------- |
| 0      | $\varepsilon$       |
| 1   | ```#```      |
| 2  | ```1```     |
| 3  |```##```      |
| 4  | ```##``` |
| 5   | ```1#```      |
| 6  | ```11```     |
| 7  |```###```      |
| 8  | ```##1``` |
| 9  | ```#1#``` |
| 10  | ```#11``` |
| 11 | ```1##```|
| 12 | ```1#1``` |
| 13 | ```11#``` |
| 14 | ```111```|
| 15 | ```####``` |

Being a bijection, this function $s$ has an inverse $s^{-1}$.

The idea now is to take all of the machinery which we have built in this notebook and use $s$ and its inverse $s^{-1}$ to transfer them to numbers.  In the other direction, we want to take all of the primitive recursive functions of numbers and transfer them to ```1#```.



### Accessing bits

We next define two functions on the natural numbers.  The first is called $|n|$, and it gives the number of characters in $s(n)$.

$$
|n| = \biggl( \mu x \leq n +1. \ (2^x  > n + 1)\biggr) - 1.
$$

$$ 
\begin{array}{lcl}
rem(a,b) &  = &  (\mu r < b).(\exists q)(a = qb + r)\\
&  = &  (\mu r < b).(\exists q \leq a)(a = qb + r)\\
\end{array}
$$

$$
(n)_{m} =\left\{
\begin{array}{ll}
0 & if\ rem(n+1,2^{m+1}) <  2^m   \\
1 & if\    rem(n+1,2^{m+1}) \geq 2^m  
\end{array}
\right.
$$


Here is a table of the values of this function for small numbers.


| $n$ | $s(n)$        | $\|n\|$ | $(n)_0$ | $(n)_1$ | $(n)_2$ |
|-----|---------------|---------|---------|---------|---------|
| 0   | $\varepsilon$ | 0       | 0       | 0       | 0      |
| 1   | ```#```         | 1       | 0       | 0       | 0       |
| 2   | ```1```       | 1       | 1       | 0       | 0       |
| 3   | ```##```      | 2       | 0       | 0       | 0       |
| 4   | ```#1```      | 2       | 1       | 0       | 0       |
| 5   | ```1#```      | 2       | 0       | 1       | 0       |
| 6   | ```11```      | 2       | 1       | 1       | 0       |
| 7   | ```###```     | 3       | 0       | 0       | 0       |
| 8   | ```##1```     | 3       | 1       | 0       | 0       |
| 9   | ```#1#```     | 3       | 0       | 1       | 0       |
| 10  | ```#11```     | 3       | 1       | 1       | 0       |
| 11  | ```1##```     | 3       | 0       | 0       | 1       |
| 12  | ```1#1```     | 3       | 1       | 0       | 1       |
| 13  | ```11#```     | 3       | 0       | 1       | 1       |
| 14  | ```111```     | 3       | 1       | 1       | 1       |
| 15  | ```####```     | 4       | 0       | 0      | 0       |
